# Chat bot creation using ANN

# Loading libraries

In [1]:
import numpy
import tflearn
import tensorflow as tf
import random
import os
import json
import pickle

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
import nltk
nltk.download('punkt')
nltk.download('popular')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\B.THUSHAR\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\B.THUSHAR\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\B.THUSHAR\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\B.THUSHAR\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\B.THUSHAR\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     

In [3]:
with open("intents1.json") as file:
    data = json.load(file)
    print(data["intents"])

try:
  with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
        
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

[{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hello, thanks for visiting', 'Good to see you again', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'Course', 'patterns': ['where i can find good online course for machine learning and data science', 'Data science', 'Machine learnng', 'online course in data science'], 'responses': ['There are many platforms to learn Data science and Machine learning course I will list some of the online platforms -Udemy,Coursera,Edx,Analytics Vidya,great learning '], 'context_set': ''}, {'tag': 'Course', 'patterns': ['where i can find good online course for machine learning and data science', 'Data science', 'Machine learnng', 'online course in data science'], 'responses': ['There are many platforms to learn Data science and Machine learning course I will list some of the online platforms -Udemy,Coursera,Edx,Analytics Vidya,great learning '], 'context_set': ''}, {'tag': 'goodbye',

In [4]:
for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

In [5]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [15]:
training.shape

(83, 104)

In [16]:
output.shape

(83, 20)

In [6]:

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)       

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)



In [7]:

training = numpy.array(training)
output = numpy.array(output)

In [8]:
with open("data.pickle", "wb") as f:
    pickle.dump((words, labels, training, output), f)

In [9]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
if os.path.exists("model.tflearn.meta"):
    model.load("model.tflearn")
else:
    model.fit(training, output, n_epoch=1000,batch_size=8, show_metric=True)
    model.save("model.tflearn")

Training Step: 10999  | total loss: 0.03619 | time: 0.086s
| Adam | epoch: 1000 | loss: 0.03619 - acc: 0.9692 -- iter: 80/83
Training Step: 11000  | total loss: 0.03264 | time: 0.095s
| Adam | epoch: 1000 | loss: 0.03264 - acc: 0.9723 -- iter: 83/83
--
INFO:tensorflow:C:\Users\B.THUSHAR\Desktop\ML projects\Chat bot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [11]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


In [13]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
#         print(results)
        print(tag)
        if results[results_index] > 0.6:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']

            print(random.choice(responses))
        else:
            print("I am not sure. Please contact SkillPractical support @support@skillpractical.com")
chat()

Start talking with the bot (type quit to stop)!
You: hi
greeting
Hello, thanks for visiting
You: quit
